In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests

In [ ]:
#una pajina dinamica, entonces Selenium
driver = webdriver.Chrome()
driver.get("https://www.argentina.gob.ar/anmat/normativa-general")

pdf_links = []

for i in range(36):
    time.sleep(5)
    
    #guardar botones
    buttons = driver.find_elements(By.CLASS_NAME, 'btn.btn-primary.btn-sm.margin-btn')
    pdf_links.extend([button.get_attribute('href') for button in buttons])
    
    #la pajina siguente
    next_button = driver.find_element(By.XPATH, '//a[@aria-label="Ir a la página siguiente"]')
    
    #no hay mas pajinas
    #if 'disabled' in next_button.get_attribute('class'):
    #    break
    
    next_button.click()

driver.quit()

In [ ]:
len(pdf_links)

In [ ]:
os.makedirs('anmat', exist_ok=True)

In [ ]:
#descargar pdf
for pdf_link in pdf_links:
    if '.pdf' in pdf_link:
        pdf_response = requests.get(pdf_link)
        file_name = os.path.join('anmat', pdf_link.split("/")[-1] + '.pdf')
        with open(file_name, 'wb') as file:
            file.write(pdf_response.content)
        print(f"{file_name}")

In [ ]:
#descargar texto
for non_pdf_link in pdf_links:
    if '.pdf' not in non_pdf_link:
        response = requests.get(non_pdf_link+'/texto')
        file_name = os.path.join('anmat', non_pdf_link.split("/")[-1] + '.txt')
        with open(file_name, 'wb') as file:
            file.write(response.content)
        print(f"{file_name}")